In [1]:
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
# from tqdm import tqdm, tqdm_notebook,tqdm_pandas
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

# =============================================================================================================

In [2]:
start_time = datetime.now()

clause_df = pd.read_excel(r".\Clauses-15.03.22.xlsx", 'Sheet2').iloc[:, :5]

In [3]:
prop_cndtn_df = pd.read_excel(r"C:\Users\Rahul\Desktop\propname wise\my_properties.xlsx", "yield").iloc[:, :9]


In [4]:
desc_cls_file = pd.read_excel(r'.\desc_replace.xlsx')


In [5]:
# char_data = pd.read_excel('.\CHAR_EXCELS\SPIR_SUN21041_25_04_2022.xls', 'Sheet3').fillna('')
char_data = pd.read_excel('165 Descriptions.xlsx', 'Sheet3').fillna('')
class_data = char_data[:450]
print(type(class_data))
print(class_data)

<class 'pandas.core.frame.DataFrame'>
   MDRM             Class                                   Long Description
0     1  CARTRIDGE, TONER  CARTRIDGE, TONER, MAGENTA, P/N 310-5738, 2000 ...


In [6]:
clauseData = clause_df
property_cndtn_data = prop_cndtn_df
classData = class_data
classData
classData.rename(columns={'PART_REF_ISSUED': 'MDRM', 'ITEM_DESC': 'Long Description', 'OBJ_QUAL': 'Class'}, inplace=True)
# classData.rename(columns={'REGISTERED_RECORD_NO': 'MDRM', 'LONG_TEXT': 'Long Description', 'CLASS': 'Class'}, inplace=True)

classData['Long Description'] = classData['Long Description'].str.upper()
classData.drop_duplicates(inplace=True, keep='first', ignore_index=True)

In [7]:
classData

,MDRM,Class,Long Description
0,1,"CARTRIDGE, TONER","CARTRIDGE, TONER, MAGENTA, P/N 310-5738, 2000 ..."


In [8]:
# start_time = datetime.now()
# clauseData = clause_data
# property_cndtn_data = property_cndtn_data
# classData = class_data
# # classData.rename(columns={'PART_REF_ISSUED': 'MDRM', 'ITEM_DESC': 'Long Description', 'OBJ_QUAL': 'Class'}, inplace=True)
# classData.rename(columns={'REGISTERED_RECORD_NO': 'MDRM', 'LONG_TEXT': 'Long Description', 'CLASS': 'Class'}, inplace=True)

# classData['Long Description'] = classData['Long Description'].str.upper()
# classData.drop_duplicates(inplace=True, keep='first', ignore_index=True)

#-------------------------------------------------------------------------------------------------------------------
# df = pd.read_excel('Is_properties_condition_working_21.03.22_rev1.xls', 'Sheet5')
# df.drop_duplicates(inplace=True, keep='first', ignore_index=True)
# print(df)
#-------------------------------------------------------------------------------------------------------------------

# classData.columns
# df = pd.read_excel('Is_properties_condition_working_21.03.22_rev1.xls', 'Sheet5')
# df = sheet5_df
# df.drop_duplicates(inplace=True, keep='first', ignore_index=True)
# print(df)
#-------------------------------------------------------------------------------------------------------------------
def cleanDescription(cls, desc_cls_file, des1):
    # -------------------------------------------------------------------------
    if cls in desc_cls_file.Class.tolist():

        cls_idx_lst = desc_cls_file.index[desc_cls_file['Class'] == cls].tolist()

        for cls_idx in cls_idx_lst:

            if desc_cls_file['Value'][cls_idx] in des1:
                print('Orig Desc: ', des1, "|", 'Modified Desc: ', des1.replace(desc_cls_file['Value'][cls_idx], desc_cls_file['Value_replace'][cls_idx]), file=print_file)
                des1 = des1.replace(desc_cls_file['Value'][cls_idx], desc_cls_file['Value_replace'][cls_idx])
        print('=' * 100, '\n', file=print_file)
    return des1

# classData.columns
def altr_des(des):
    des = re.sub(r'\"', ' INCH ', des)
    des = re.sub(r' +',' ',des)
    des = re.sub(r'\n',' ',des)
    des = re.sub(r'( ,|, )',',',des)
    des = re.sub(r'( :|: )',':',des)
    des = re.sub(r'( ;|; )',';',des)
    des = re.sub(r'( -|- | - )','-',des)
    des = re.sub(r'( _|_ | _ )','_',des)
    des = re.sub(r'( \/|\/ )','/',des)
    x = des.upper()

#     print(x)
    return x

# =============================================================================================================
print_file = open('print_check.txt', 'w')


def property_extraction(des,cls):

    des = altr_des(des)
    des = cleanDescription(cls, desc_cls_file, des)
    prop_name = []

    des1 = des
    des2 = des
    des3 = des

    for pcls,prop,uom,rule_uom,cndtn_chk,ptype in zip(property_cndtn_data['CLASS'],
                                                    property_cndtn_data['PROPERTY'],
                                                    property_cndtn_data['UOM'],
                                                    property_cndtn_data['RULE_UOM'],
                                                    property_cndtn_data['CONDITION_CHECK'],
                                                    property_cndtn_data['TYPE']):

        if cls == pcls and ptype == 'REGEXP':
            # -------------Cleaning Description----------------
#             print(prop)
#             print('condition_check', str(cndtn_chk))
#             print(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1))
#             print("=" * 25)
            # -------------------------------------------------

            if re.search(r'\b{}\b'.format(cndtn_chk),des1) != None:

                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip(),uom,rule_uom])
                # prop1.append(prop)
                # prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
                # prop1.append(uom)
                # prop1.append(rule_uom)
                # print(cndtn_chk)
                des1 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())), '', des1)

        elif cls == pcls and ptype == 'CLAUSE':

            # -------------Cleaning Description----------------
#             des2 = cleanDescription(cls, desc_cls_file, des2)
            # -------------------------------------------------

            for cl_name,cl_val,o_des in zip(clauseData['CLAUSE_NAME'],clauseData['CLAUSE_VALUE'], clauseData['O_DESCRIPTOR']):
#                 if (cndtn_chk == cl_name) and ((pcls == o_des) | (pcls == 'ALL')):
#                 if (pcls == 'ALL'):
#                     print(pcls,o_des,cndtn_chk)
                if (cndtn_chk == cl_name) and (pcls == o_des or o_des == 'ALL'):

                    if cl_val in des2:

                        if re.search(r'\b{}\b'.format(str(cl_val)),des2) != None:
                            prop_name.append([prop,re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip(),uom,rule_uom])
        #                             prop1.append(prop)
        #                             prop1.append(re.search(r'\b{}\b'.format(str(cl_val)),des1).group().strip())
        #                             prop1.append(uom)
        #                             prop1.append(rule_uom)
                            des2 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip())), '', des2)



        elif cls == pcls and ptype == 'NORMAL':
            # -------------Cleaning Description----------------
#             des3 = cleanDescription(cls, desc_cls_file, des3)
            # -------------------------------------------------
            if re.search(r'\b{}\b'.format(str(cndtn_chk)),des3) != None:
                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip(),uom,rule_uom])
#                 prop1.append(prop)
#                 prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
#                 prop1.append(uom)
#                 prop1.append(rule_uom)
                des3 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip())), '', des3)
        else:
            pass

    prop_list = []
    for i in prop_name:
        if i not in prop_list:
            prop_list.append(i)

    # print(prop_list)
    # print(des1, ' | ', des)
    # return prop_list, des
    return prop_list

In [10]:
property_extraction('FUSE; 690 VAC, 500 A, APPLICATION: SRV', 'FUSE, CARTRIDGE')

[['CURRENT RATING', ',500 A,', 'AMPERE', 'A']]

In [9]:
def property_extraction2(des,cls):

    des = altr_des(des)
    des = cleanDescription(cls, desc_cls_file, des)
    prop_name = []

    des1 = des
    des2 = des
    des3 = des

    for pcls,prop,uom,rule_uom,cndtn_chk,ptype in zip(property_cndtn_data['CLASS'],
                                                    property_cndtn_data['PROPERTY'],
                                                    property_cndtn_data['UOM'],
                                                    property_cndtn_data['RULE_UOM'],
                                                    property_cndtn_data['CONDITION_CHECK'],
                                                    property_cndtn_data['TYPE']):

        if cls == pcls and ptype == 'REGEXP':
            # -------------Cleaning Description----------------
#             print(prop)
#             print('condition_check', str(cndtn_chk))
#             print(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1))
#             print("=" * 25)
            # -------------------------------------------------

            if re.search(r'\b{}\b'.format(cndtn_chk),des1) != None:

                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip(),uom,rule_uom])
                # prop1.append(prop)
                # prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
                # prop1.append(uom)
                # prop1.append(rule_uom)
                # print(cndtn_chk)
                des1 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())), '', des1)

        elif cls == pcls and ptype == 'CLAUSE':

            # -------------Cleaning Description----------------
#             des2 = cleanDescription(cls, desc_cls_file, des2)
            # -------------------------------------------------

            for cl_name,cl_val,o_des in zip(clauseData['CLAUSE_NAME'],clauseData['CLAUSE_VALUE'], clauseData['O_DESCRIPTOR']):
#                 if (cndtn_chk == cl_name) and ((pcls == o_des) | (pcls == 'ALL')):
#                 if (pcls == 'ALL'):
#                     print(pcls,o_des,cndtn_chk)
                if (cndtn_chk == cl_name) and (pcls == o_des or o_des == 'ALL'):

                    if cl_val in des2:

                        if re.search(r'\b{}\b'.format(str(cl_val)),des2) != None:
                            prop_name.append([prop,re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip(),uom,rule_uom])
        #                             prop1.append(prop)
        #                             prop1.append(re.search(r'\b{}\b'.format(str(cl_val)),des1).group().strip())
        #                             prop1.append(uom)
        #                             prop1.append(rule_uom)
                            des2 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip())), '', des2)



        elif cls == pcls and ptype == 'NORMAL':
            # -------------Cleaning Description----------------
#             des3 = cleanDescription(cls, desc_cls_file, des3)
            # -------------------------------------------------
            if re.search(r'\b{}\b'.format(str(cndtn_chk)),des3) != None:
                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip(),uom,rule_uom])
#                 prop1.append(prop)
#                 prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
#                 prop1.append(uom)
#                 prop1.append(rule_uom)
                des3 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip())), '', des3)
        else:
            pass

    prop_list = []
    for i in prop_name:
        if i not in prop_list:
            prop_list.append(i)

    # print(prop_list)
    # print(des1, ' | ', des)
    # return prop_list, des
    return prop_list

# =============================================================================================================

# classData['Properties'] = classData.apply(lambda x: property_extraction(x['Long Description'],x.Class)[0],axis=1)


In [11]:
property_extraction2('MCCB, 4P, 500 A, VL630, 500 V/30 KADC/25KAAC, 415V/45KAAC, DESIGN: Q005, PART NO: 530-0861, EQUIPMENT NAME: UPS, EQUIPMENT MANUFACTURER NAME: GUTOR', 'BREAKER, CIRCUIT')

[['ELECTRICAL POLE QUANTITY', '4P,', nan, nan],
 ['TYPE', 'MCCB', 'NOT APPLICABLE', nan],
 ['VOLTAGE', ',500 V', 'VOLT', '(VOLT|V)'],
 ['APPLICATION',
  ',EQUIPMENT NAME:UPS,',
  nan,
  '(EQUIPMENT NAME|EQUIPMENTNAME|EQIMN|\\s|X|,|)'],
 ['INTERRUPT CAPACITY', '30 KADC', nan, nan],
 ['CURRENT', ',500 A', 'AMPERE', '(AMPS|AMP|A)']]

# =============================================================================================================
=============================================================================================================
=============================================================================================================

In [1]:
# !conda activate python385_env


In [7]:
!


python: can't open file 'env': [Errno 2] No such file or directory


In [2]:
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
# from tqdm import tqdm, tqdm_notebook,tqdm_pandas
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()


In [3]:
clauseData = pd.read_excel("Clauses-15.03.22.xlsx", 'Sheet2').iloc[:,:5]
property_cndtn_data = pd.read_excel("Is_properties_condition_working.xls", "Sheet4").iloc[:,:9]
# property_cndtn_data = pd.read_excel(r"C:\Users\Rahul\Desktop\propname wise\my_properties.xlsx", "yield").iloc[:,:9]
desc_cls_file = pd.read_excel('desc_replace.xlsx')
classData = pd.read_excel("165 Descriptions.xlsx", "Sheet3")
classData['Long Description'] = classData['Long Description'].str.upper()
classData.drop_duplicates(inplace = True, keep=False, ignore_index=True)
classData.columns

Index(['MDRM', 'Class', 'Long Description'], dtype='object')

In [6]:
import datetime
start_time = datetime.now()
# clauseData = clause_data
property_cndtn_data = property_cndtn_data
# classData = class_data
classData.rename(columns={'PART_REF_ISSUED': 'MDRM', 'ITEM_DESC': 'Long Description', 'OBJ_QUAL': 'Class'}, inplace=True)
# classData.rename(columns={'REGISTERED_RECORD_NO': 'MDRM', 'LONG_TEXT': 'Long Description', 'CLASS': 'Class'}, inplace=True)

classData['Long Description'] = classData['Long Description'].str.upper()
classData.drop_duplicates(inplace=True, keep='first', ignore_index=True)

NameError: name 'property_cndtn_data' is not defined

In [ ]:
classData

In [5]:

#-------------------------------------------------------------------------------------------------------------------
def cleanDescription(cls, desc_cls_file, des1):
    # -------------------------------------------------------------------------
    if cls in desc_cls_file.Class.tolist():

        cls_idx_lst = desc_cls_file.index[desc_cls_file['Class'] == cls].tolist()

        for cls_idx in cls_idx_lst:

            if desc_cls_file['Value'][cls_idx] in des1:
                print('Orig Desc: ', des1, "|", 'Modified Desc: ', des1.replace(desc_cls_file['Value'][cls_idx], desc_cls_file['Value_replace'][cls_idx]), file=print_file)
                des1 = des1.replace(desc_cls_file['Value'][cls_idx], desc_cls_file['Value_replace'][cls_idx])
        print('=' * 100, '\n', file=print_file)
    return des1

# classData.columns
def altr_des(des):
    des = re.sub(r'\"','INCH',des)
    des = re.sub(r' +',' ',des)
    des = re.sub(r'\n',' ',des)
    des = re.sub(r'( ,|, )',', ',des)
    des = re.sub(r'( :|: )',': ',des)
    des = re.sub(r'( ;|; )','; ',des)
    des = re.sub(r'( -|- | - )','-',des)
    des = re.sub(r'( _|_ | _ )','_',des)
    des = re.sub(r'( \/|\/ )','/',des)
    x = des.upper()

#     print(x)
    return x


In [14]:
def property_extraction(des,cls):

    des = altr_des(des)
    des = cleanDescription(cls, desc_cls_file, des)
    prop_name = []

    des1 = des
    des2 = des
    des3 = des
    
    print(des)
    for pcls,prop,uom,rule_uom,cndtn_chk,ptype in zip(property_cndtn_data['CLASS'],
                                                    property_cndtn_data['PROPERTY'],
                                                    property_cndtn_data['UOM'],
                                                    property_cndtn_data['RULE_UOM'],
                                                    property_cndtn_data['CONDITION_CHECK'],
                                                    property_cndtn_data['TYPE']):

        if cls == pcls and ptype == 'REGEXP':
            # -------------Cleaning Description----------------
#             print(prop)
#             print('condition_check', str(cndtn_chk))
#             print(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1))
#             print("=" * 25)
            # -------------------------------------------------

            if re.search(r'\b{}\b'.format(cndtn_chk),des1) != None:

                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip(),uom,rule_uom])
                # prop1.append(prop)
                # prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
                # prop1.append(uom)
                # prop1.append(rule_uom)
                # print(cndtn_chk)
                des1 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())), '', des1)
#                 print(des1)

        if cls == pcls and ptype == 'CLAUSE':

            # -------------Cleaning Description----------------
#             des2 = cleanDescription(cls, desc_cls_file, des2)
            # -------------------------------------------------
            print(des2)
            for cl_name,cl_val,o_des in zip(clauseData['CLAUSE_NAME'],clauseData['CLAUSE_VALUE'], clauseData['O_DESCRIPTOR']):
#                 if (cndtn_chk == cl_name) and ((pcls == o_des) | (pcls == 'ALL')):
#                 if (pcls == 'ALL'):
#                     print(pcls,o_des,cndtn_chk)
                
                if (cndtn_chk == cl_name) and (pcls == o_des or o_des == 'ALL'):
#                     print(cl_val)
                    if cl_val in des2:
#                         print("============",des2,"================")
#                         print("============",cl_val,"================")
                        if re.search(r'\b{}\b'.format(str(cl_val)),des2) != None:
                            prop_name.append([prop,re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip(),uom,rule_uom])
        #                             prop1.append(prop)
        #                             prop1.append(re.search(r'\b{}\b'.format(str(cl_val)),des1).group().strip())
        #                             prop1.append(uom)
        #                             prop1.append(rule_uom)
                            des2 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cl_val)),des2).group().strip())), '', des2)



        elif cls == pcls and ptype == 'NORMAL':
            # -------------Cleaning Description----------------
#             des3 = cleanDescription(cls, desc_cls_file, des3)
            # -------------------------------------------------
            if re.search(r'\b{}\b'.format(str(cndtn_chk)),des3) != None:
                prop_name.append([prop,re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip(),uom,rule_uom])
#                 prop1.append(prop)
#                 prop1.append(re.search(r'\b{}\b'.format(str(cndtn_chk)),des1).group().strip())
#                 prop1.append(uom)
#                 prop1.append(rule_uom)
                des3 = re.sub(re.escape(str(re.search(r'\b{}\b'.format(str(cndtn_chk)),des3).group().strip())), '', des3)
        else:
            pass

    prop_list = []
    for i in prop_name:
        if i not in prop_list:
            prop_list.append(i)

    # print(prop_list)
    # print(des1, ' | ', des)
    # return prop_list, des
    return prop_list

# =============================================================================================================

# classData['Properties'] = classData.apply(lambda x: property_extraction(x['Long Description'],x.Class)[0],axis=1)


In [15]:
classData['Properties'] = classData.apply(lambda x: property_extraction(x['Long Description'],x.Class),axis=1)


classData.fillna('', inplace = True, axis = 1)

WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, , APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, , APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, , APPLICATION: HEAVY DUTY MOTOR  
WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, , APPLICATION:   
WASHER, FLAT, SIZE 1INCH, 1.0

In [16]:
classData['Prop_Name'] = ''
classData['Prop_Val'] = ''
classData['uom'] = ''
classData['r_uom'] = ''

In [17]:
classData

,MDRM,Class,Long Description,Properties,Prop_Name,Prop_Val,uom,r_uom
0,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",,,,
1,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",,,,


In [18]:
classData['Long Description'][0]

'WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 INCH X THK 0.165 INCH, ASMEB18.22.1, APPLICATION: HEAVY DUTY MOTOR\n\n'

In [19]:
classData['Properties'][0]

[['STANDARD', 'ASMEB18.22.1', nan, nan],
 ['INSIDE DIAMETER',
  'ID 1.063 INCH',
  'INCH',
  '(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\\")'],
 ['OUTSIDE DIAMETER',
  'OD 2.500 INCH',
  'INCH',
  '(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\\")'],
 ['THICKNESS', 'THK 0.165 INCH', 'INCH', '(THICKNESS|THK|T|INCH|IN|\\")'],
 ['APPLICATION', 'HEAVY DUTY MOTOR', nan, nan],
 ['NOMINAL SIZE', 'SIZE:1 INCH', 'INCH', '(INCH|IN|\\"|NOMINAL SIZE|SIZE)']]

In [21]:
classData['Long Description'][1]

'WASHER, FLAT, SIZE 1", 1.063 " ID X 2.500 " OD X 0.165 " THK, ASMEB18.22.1, APPLICATION HEAVY DUTY PUMP FOR MIL SECTION OPEN AREA\n'

In [22]:
classData['Properties'][1]

[['STANDARD', 'ASMEB18.22.1', nan, nan],
 ['INSIDE DIAMETER',
  '1.063 INCH ID',
  'INCH',
  '(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\\")'],
 ['OUTSIDE DIAMETER',
  '2.500 INCH OD',
  'INCH',
  '(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\\")'],
 ['THICKNESS', '0.165 INCH THK', 'INCH', '(THICKNESS|THK|T|INCH|IN|\\")'],
 ['APPLICATION', 'HEAVY DUTY PUMP FOR MIL SECTION OPEN AREA', nan, nan],
 ['NOMINAL SIZE', 'SIZE 1INCH', 'INCH', '(INCH|IN|\\"|NOMINAL SIZE|SIZE)']]

In [10]:
property_extraction('FUSE; VOLTAGE RATING: 690 VAC; CURRENT RATING: 500 A; APPLICATION: SRV', 'FUSE, CARTRIDGE')


FUSE;VOLTAGE RATING:690 VAC;CURRENT RATING;APPLICATION:SRV
FUSE;VOLTAGE RATING;CURRENT RATING;APPLICATION:SRV


[['CURRENT RATING', ':500 A', 'AMPERE', 'A'],
 ['VOLTAGE RATING', ':690 VAC', 'VOLT ALTERNATING CURRENT', '(VAC|V AC)']]

In [11]:
property_extraction('FUSE; 690 VAC, 500 A, APPLICATION: SRV', 'FUSE, CARTRIDGE')

FUSE;690 VAC,APPLICATION:SRV
FUSE;,APPLICATION:SRV


[['CURRENT RATING', ',500 A', 'AMPERE', 'A'],
 ['VOLTAGE RATING', '690 VAC', 'VOLT ALTERNATING CURRENT', '(VAC|V AC)']]

In [10]:
property_extraction('FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM', 'FUSE, CARTRIDGE')

FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM
============ FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM ================
============ PE ================
============ FUSE; TYPE: FAST SERIES IN GLASS; SIZE; 5 X 20 MM ================
============ GLASS ================
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
============ FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM ================
============ IE ================
============ FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM ================
============ US ================
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM
============ FUSE; TYPE: FAST SERIES IN ; SIZE; 5 X 20 MM ================
============ FA

[['CASE MATERIAL', 'GLASS', nan, nan],
 ['TYPE', 'FAST', nan, nan],
 ['CASE MATERIAL', 'GLASS', 'NOT APPLICABLE', nan]]

In [9]:
property_extraction('FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA', 'FUSE, CARTRIDGE')

FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA
FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA
FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA
============ FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA ================
============ PE ================
FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA
FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT RATING: 120 KA
============ FUSE, HRC; TYPE: GENERAL PURPOSE, CYLINDRICAL; SIZE: 10 X 38 MM; VOLTAGE RATING: 500 V; CURRENT RATING: 4 A; INTERRUPT R

[['APPLICATION', 'CYLINDRICAL', nan, nan]]

In [12]:
property_extraction('FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC', 'FUSE, CARTRIDGE') 

FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC
FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC
FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC
============ FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC ================
============ PE ================
FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC
FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC
============ FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; 

[['STANDARD', 'CC', nan, nan],
 ['TYPE', 'FAST ACTING-I, CURRENT LIMITING', nan, nan]]

In [17]:
'FAST ACTING-I, CURRENT LIMITING' in qq

True

In [ ]:
FAST ACTING-I, CURRENT LIMITING

In [36]:
def altr_des(des):
    des = re.sub(r' +',' ',des)
    des = re.sub(r'\n',' ',des)
    des = re.sub(r'( ,|, )',', ',des)
    des = re.sub(r'( :|: )',': ',des)
    des = re.sub(r'( ;|; )','; ',des)
    des = re.sub(r'( -|- | - )','-',des)
    des = re.sub(r'( _|_ | _ )','_',des)
    des = re.sub(r'( \/|\/ )','/',des)
    x = des.upper()

#     print(x)
    return x

altr_des(qq)

'FUSE, HRC; TYPE: FAST ACTING-I, CURRENT LIMITING; SIZE: 10.3 X 38.1 MM; VOLTAGE: 600 VAC; CURRENT: 8 A; INTERRUPT RATING: 200000 A; CLASS: CC'

In [23]:


classData['Quantity'] = classData['Properties'].apply(lambda x : len(x))
classData.reset_index(drop=True, inplace=True)

classData_z = classData[classData['Quantity'] == 0]
classData_z.reset_index(drop= True, inplace = True)

classData_n = classData[classData['Quantity'] != 0]
classData_n.reset_index(drop= True, inplace = True)

classData_new = classData_n.loc[classData_n.index.repeat(classData_n.Quantity)].reset_index(drop=True)
classData_new.to_excel('classData_new_Error.xlsx')

In [24]:
classData['Properties'][3:4][3]

KeyError: 3

In [25]:

group = classData_new.groupby(['MDRM'])

classData_new1 = pd.DataFrame(columns=['MDRM', 'Class', 'Long Description','Properties','Prop_Name','Prop_Val','uom','r_uom',
                                    'Quantity'])

for i in (classData_new['MDRM'].unique()):

    batch = group.get_group(i)
    batch.reset_index(drop = True, inplace = True)

    for j in range(len(batch)):
#         print(batch['Properties'])

        batch['Prop_Name'][j] = batch['Properties'][j][j][0]
        batch['Prop_Val'][j] = batch['Properties'][j][j][1]
        batch['uom'][j] = batch['Properties'][j][j][2]
        batch['r_uom'][j] = batch['Properties'][j][j][3]
#         print(batch)
#         print("="*40)

    classData_new1 = pd.concat([classData_new1, batch], ignore_index = True)

# classData_new1.columns
classData_new1.fillna('', inplace = True, axis = 1)
classData_new1.to_excel('checking1.xlsx', index = False)

<ipython-input-25-eac4a65139b9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Prop_Name'][j] = batch['Properties'][j][j][0]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-25-eac4a65139b9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Prop_Val'][j] = batch['Properties'][j

In [26]:
classData_new1

,MDRM,Class,Long Description,Properties,Prop_Name,Prop_Val,uom,r_uom,Quantity
0,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
1,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,ID 1.063 INCH,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
2,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,OD 2.500 INCH,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
3,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,THK 0.165 INCH,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6
4,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",APPLICATION,HEAVY DUTY MOTOR,,,6
5,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",NOMINAL SIZE,SIZE:1 INCH,INCH,"(INCH|IN|\""|NOMINAL SIZE|SIZE)",6
6,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
7,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,1.063 INCH ID,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
8,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,2.500 INCH OD,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
9,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,0.165 INCH THK,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6


In [27]:

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: re.sub(r'^[\.\,\:\-\/()+xX]{1,6}|[\.\,\:\-\/()+xX]{1,6}$','',x['Prop_Val']), axis = 1)

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: re.sub(r'^[\,\:]{1,2}|[\,\:]{1,2}$','',x['Prop_Val']), axis = 1)

classData_new1['Prop_Val'] = classData_new1['Prop_Val'].apply(lambda x: re.sub(r'[\[\]\']', '', str(x)))

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: x['Prop_Val'].replace(x['uom'], ''), axis = 1)

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: x['Prop_Val'].replace(x['Prop_Name'], ''), axis = 1)

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: x['Prop_Val'].replace(x['r_uom'], ''), axis = 1)

classData_new1['Prop_Val'] = classData_new1.apply(lambda x: re.sub(x['r_uom'], '', x['Prop_Val']),axis=1)

#     ==============================================================================================================
classData_new1.to_excel('checking2.xlsx', index = False)

In [28]:
classData_new1

,MDRM,Class,Long Description,Properties,Prop_Name,Prop_Val,uom,r_uom,Quantity
0,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
1,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,1.063,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
2,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,2.500,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
3,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,0.165,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6
4,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",APPLICATION,HEAVY DUTY MOTOR,,,6
5,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",NOMINAL SIZE,:1,INCH,"(INCH|IN|\""|NOMINAL SIZE|SIZE)",6
6,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
7,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,1.063,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
8,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,2.500,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
9,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,0.165,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6


In [29]:

group1 = classData_new1.groupby(['MDRM'])

final_df = pd.DataFrame()
# classData_new1

for i in (classData_new1['MDRM'].unique()):
    batch1 = group1.get_group(i)
    batch1['Prop_Val'] = batch1.groupby(['Prop_Name'])['Prop_Val'].transform(lambda x : ' ; '.join(x))
    batch1['uom'] = batch1.groupby(['Prop_Name'])['uom'].transform(lambda x : ','.join(x))
    batch1.drop_duplicates(subset = ['Prop_Name', 'Prop_Val', 'uom'], inplace = True, keep = 'first')
    final_df = pd.concat([final_df, batch1], ignore_index = True)

#     ==============================================================================================================
final_df.to_excel('checking2.xlsx', index = False)

<ipython-input-29-beb4d22bbaf2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch1['Prop_Val'] = batch1.groupby(['Prop_Name'])['Prop_Val'].transform(lambda x : ' ; '.join(x))
<ipython-input-29-beb4d22bbaf2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch1['uom'] = batch1.groupby(['Prop_Name'])['uom'].transform(lambda x : ','.join(x))
<ipython-input-29-beb4d22bbaf2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [30]:
final_df

,MDRM,Class,Long Description,Properties,Prop_Name,Prop_Val,uom,r_uom,Quantity
0,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
1,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,1.063,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
2,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,2.500,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
3,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,0.165,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6
4,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",APPLICATION,HEAVY DUTY MOTOR,,,6
5,1,"WASHER, FLAT","WASHER, SIZE:1 INCH, ID 1.063 INCH X OD 2.500 ...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",NOMINAL SIZE,:1,INCH,"(INCH|IN|\""|NOMINAL SIZE|SIZE)",6
6,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",STANDARD,ASMEB18.22.1,,,6
7,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",INSIDE DIAMETER,1.063,INCH,"(INSIDE DIAMETER|INSIDE DIA|ID|MM|INCH|IN|\"")",6
8,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",OUTSIDE DIAMETER,2.500,INCH,"(OUTSIDE DIAMETER|OUTSIDE DIA|OD|INCH|IN|\"")",6
9,2,"WASHER, FLAT","WASHER, FLAT, SIZE 1"", 1.063 "" ID X 2.500 "" OD...","[[STANDARD, ASMEB18.22.1, nan, nan], [INSIDE D...",THICKNESS,0.165,INCH,"(THICKNESS|THK|T|INCH|IN|\"")",6


In [ ]:

    # classData_new1.columns
    final_df.fillna('', inplace = True, axis = 1)

    final_df['Prop_Val'] = final_df.apply(lambda x: re.sub(r'^[\.\,\;\:\-\/()+]{1,3}|[\.\,\:\;\-\/()+]{1,3}$','',x['Prop_Val']), axis = 1)

    final_df['Prop_Val'] = final_df['Prop_Val'].apply(lambda x: re.sub(r'[\[\]\']', '', str(x).strip()))

    final_df['uom'] = final_df['uom'].apply(lambda x: ','.join(list(pd.unique(x.strip().split(',')))))

    final_df['uom'] = final_df.apply(lambda x: re.sub(r'^,|,$','',x['uom']), axis = 1)

    final_df.to_excel('checking3.xlsx', index = False)

    final_df = pd.concat([classData_z, final_df], ignore_index = True)
    final_df.fillna('', inplace = True, axis = 1)
    final_df.to_excel('checking4.xlsx', index = False)


    def long_des_exclude(batch):

        des = batch['Long Description'].unique()
        #         print(des)

        column_values = batch[["Class", "Prop_Name", "Prop_Val", "uom"]].values.ravel()
        unique_values = pd.unique(column_values)
        #         print(unique_values)
        q_val = []
        for k in unique_values:
            q_val.append([q for q in re.split(r'[\s,]', k)])
        q_val = sum(q_val, [])
        #         print('q_val', q_val, type(q_val))

        #         print('unique_values',unique_values)
        new_des = ' '.join(des)
        #         print('joined des', new_des)
        fft = new_des
        for i in q_val:
            #         val = re.split(r'[\s,]', i)
            fft = re.sub(re.escape(i), '', fft)
        # print('fft',fft)
        return fft

    final_df2 = pd.DataFrame()

    group2 = final_df.groupby(['MDRM'])
    for i in (final_df['MDRM'].unique()):
        batch2 = group2.get_group(i)
        batch2.reset_index(drop=True, inplace=True)
        batch2['LONG_DESCRIPTION_EXCLUDE'] = batch2.apply(lambda x: long_des_exclude(batch2), axis=1)
        final_df2 = pd.concat([final_df2, batch2], ignore_index=True)

    # final_df2.drop(['Properties', 'Quantity', 'r_uom'], axis=1, inplace=True)

else:
    final_df2 = classData
    # final_df2.drop(['Properties', 'Quantity', 'r_uom'], axis=1, inplace=True)

# print(name)
# print(val)
final_df2['uom'] = final_df2['uom'].replace('', 'N/A')
final_df2.to_excel('output_char_25.05.22.xlsx', index=False)

end_time = datetime.now()
print("Time taken : {}".format(end_time - start_time))
print(final_df2)
return final_df2.fillna('')

except Exception as e:
# print("Exception : ", e)
return "Please check: "+str(e)
# return final_df
# return final_df2


char_allocation()



# dff['Long Description_r'] = dff.apply(lambda x: x['Long Description'].replace(x['Class'], '').replace(x['Prop_Name'], '').replace(x['Prop_Val'], '').replace(x['uom'], ''), axis = 1)

In [22]:
import emojis
emojis.encode('warning :smile: :raised_eyebrow:')

'warning 😄 🤨'

In [5]:
emojis.encode('This is a message with emojis :smile: :snake:')


'This is a message with emojis 😄 🐍'

In [20]:
emojis.decode("decoding the message using python 🤨🙃")

'decoding the message using python :raised_eyebrow::upside_down_face:'

In [23]:
emojis.get('getting the 🤨🙃emojis 🐍from the data  😄 🤨')

{'🐍', '😄', '🙃', '🤨'}

In [27]:
emojis.count('getting the 🤨🙃emojis 🐍from the data  😄 🤨', unique = True)

4

In [39]:
emojis.db.get_emoji_aliases()

{':grinning:': '😀',
 ':smiley:': '😃',
 ':smile:': '😄',
 ':grin:': '😁',
 ':laughing:': '😆',
 ':satisfied:': '😆',
 ':sweat_smile:': '😅',
 ':rofl:': '🤣',
 ':joy:': '😂',
 ':slightly_smiling_face:': '🙂',
 ':upside_down_face:': '🙃',
 ':wink:': '😉',
 ':blush:': '😊',
 ':innocent:': '😇',
 ':smiling_face_with_three_hearts:': '🥰',
 ':heart_eyes:': '😍',
 ':star_struck:': '🤩',
 ':kissing_heart:': '😘',
 ':kissing:': '😗',
 ':relaxed:': '☺️',
 ':kissing_closed_eyes:': '😚',
 ':kissing_smiling_eyes:': '😙',
 ':smiling_face_with_tear:': '\U0001f972',
 ':yum:': '😋',
 ':stuck_out_tongue:': '😛',
 ':stuck_out_tongue_winking_eye:': '😜',
 ':zany_face:': '🤪',
 ':stuck_out_tongue_closed_eyes:': '😝',
 ':money_mouth_face:': '🤑',
 ':hugs:': '🤗',
 ':hand_over_mouth:': '🤭',
 ':shushing_face:': '🤫',
 ':thinking:': '🤔',
 ':zipper_mouth_face:': '🤐',
 ':raised_eyebrow:': '🤨',
 ':neutral_face:': '😐',
 ':expressionless:': '😑',
 ':no_mouth:': '😶',
 ':smirk:': '😏',
 ':unamused:': '😒',
 ':roll_eyes:': '🙄',
 ':grimacing:': '😬',

In [44]:
emojis.db.get_categories()

{'Activities',
 'Animals & Nature',
 'Flags',
 'Food & Drink',
 'Objects',
 'People & Body',
 'Smileys & Emotion',
 'Symbols',
 'Travel & Places'}

In [47]:
emojis.db.get_tags()

{'911',
 'accessibility',
 'achoo',
 'airport',
 'alphabet',
 'amazed',
 'america',
 'angel',
 'angry',
 'announcement',
 'annoyed',
 'applause',
 'approve',
 'archery',
 'attack',
 'aubergine',
 'autumn',
 'award',
 'bag',
 'bags',
 'bang',
 'barf',
 'basketball',
 'bath',
 'bawling',
 'beach',
 'beach_umbrella',
 'beauty',
 'bicep',
 'bicycle',
 'billiards',
 'birthday',
 'blind',
 'block',
 'blow',
 'blown',
 'blush',
 'boom',
 'bottle',
 'bouldering',
 'breakfast',
 'british',
 'bronze',
 'bubbly',
 'bug',
 'bunny',
 'burger',
 'burma',
 'burn',
 'bury',
 'business',
 'cafe',
 'calendar',
 'call',
 'camping',
 'canada',
 'celebration',
 'cheers',
 'chef',
 'chicken',
 'child',
 'china',
 'chocolates',
 'chop',
 'christmas',
 'cigarette',
 'classy',
 'cloud',
 'coder',
 'cold',
 'college',
 'comment',
 'confused',
 'console',
 'contest',
 'controller',
 'cool',
 'cop',
 'couple',
 'crap',
 'cream',
 'crown',
 'cruise',
 'crush',
 'cry',
 'curry',
 'cut',
 'cutlery',
 'dad',
 'dancer

In [59]:
sign = -1 if ((-7 < 0) ^  (3 < 0)) else 1
sign

-1

In [70]:
sorted(([8,6,1] , [8, 6, 9], [3, 2, 1]))

[[3, 2, 1], [8, 6, 1], [8, 6, 9]]

In [74]:
def findMedianSortedArrays(nums1, nums2):
    a, b = sorted((nums1, nums2), key=len)
    print(a, b)
    return a



nums1 = [8,6,9]
nums2 = [3,2,1]
findMedianSortedArrays(nums1, nums2)

[8, 6, 9] [3, 2, 1]


[8, 6, 9]

In [95]:
nums = [1,2,3,4]
def acc(nums):
    list1 = []
    j = 0
    for i in range(len(nums)):
        print(nums[i])
        list1.append(nums[i] + j)
        j = j + nums[i]
    return list1
acc(nums) 

1
2
3
4


[1, 3, 6, 10]

In [107]:
import pandas as pd
df = pd.DataFrame()

In [108]:
df['val'] = None
df

,val


In [111]:
import re
if (re.search('9', 'V-RING VS-30 NBR 80')):
    df['val'] = 'yes'
else:
    df['val'] = 'no'

In [112]:
df

,val
